In [2]:
# Dependencies
import pandas as pd
import tensorflow as tf
import boto3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


health_df = pd.read_csv('../DATA/Resources/2015.csv')



In [3]:
health_df = health_df.dropna()

health_df

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3


In [4]:
health_df.columns.to_list()

['_STATE',
 'FMONTH',
 'IDATE',
 'IMONTH',
 'IDAY',
 'IYEAR',
 'DISPCODE',
 'SEQNO',
 '_PSU',
 'CTELENUM',
 'PVTRESD1',
 'COLGHOUS',
 'STATERES',
 'CELLFON3',
 'LADULT',
 'NUMADULT',
 'NUMMEN',
 'NUMWOMEN',
 'CTELNUM1',
 'CELLFON2',
 'CADULT',
 'PVTRESD2',
 'CCLGHOUS',
 'CSTATE',
 'LANDLINE',
 'HHADULT',
 'GENHLTH',
 'PHYSHLTH',
 'MENTHLTH',
 'POORHLTH',
 'HLTHPLN1',
 'PERSDOC2',
 'MEDCOST',
 'CHECKUP1',
 'BPHIGH4',
 'BPMEDS',
 'BLOODCHO',
 'CHOLCHK',
 'TOLDHI2',
 'CVDINFR4',
 'CVDCRHD4',
 'CVDSTRK3',
 'ASTHMA3',
 'ASTHNOW',
 'CHCSCNCR',
 'CHCOCNCR',
 'CHCCOPD1',
 'HAVARTH3',
 'ADDEPEV2',
 'CHCKIDNY',
 'DIABETE3',
 'DIABAGE2',
 'SEX',
 'MARITAL',
 'EDUCA',
 'RENTHOM1',
 'NUMHHOL2',
 'NUMPHON2',
 'CPDEMO1',
 'VETERAN3',
 'EMPLOY1',
 'CHILDREN',
 'INCOME2',
 'INTERNET',
 'WEIGHT2',
 'HEIGHT3',
 'PREGNANT',
 'QLACTLM2',
 'USEEQUIP',
 'BLIND',
 'DECIDE',
 'DIFFWALK',
 'DIFFDRES',
 'DIFFALON',
 'SMOKE100',
 'SMOKDAY2',
 'STOPSMK2',
 'LASTSMK2',
 'USENOW3',
 'ALCDAY5',
 'AVEDRNK2',
 'DRNK3GE

In [5]:
# Extract data we care about
clean_health_df = health_df[[
                                   
                                   '_MICHD', # Respondents that have ever reported having coronary heart disease (CHD) or myocardial infarction (MI)
                                   '_RFHYPE5', # Adults who have been told they have high blood pressure by a doctor, nurse, or other health professional
                                   'TOLDHI2', # Have you EVER been told by a doctor, nurse or other health professional that your blood cholesterol is high?
                                   '_BMI5', # Body Mass Index (BMI)
                                   'SMOKE100', # Have you smoked at least 100 cigarettes in your entire life?
                                   'CVDSTRK3', # (Ever told) you had a stroke.
                                   'GENHLTH', # Would you say that in general your health is:
                                   'PHYSHLTH', # Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good?
                                   'SEX', # Indicate sex of respondent
                                   '_AGEG5YR', # Fourteen-level age category
                                    'MENTHLTH' # Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good?
                                   ]]

print(clean_health_df.shape)
clean_health_df.head(20)

(0, 11)


,_MICHD,_RFHYPE5,TOLDHI2,_BMI5,SMOKE100,CVDSTRK3,GENHLTH,PHYSHLTH,SEX,_AGEG5YR,MENTHLTH


In [6]:
clean_health_df.count()

_MICHD      0
_RFHYPE5    0
TOLDHI2     0
_BMI5       0
SMOKE100    0
CVDSTRK3    0
GENHLTH     0
PHYSHLTH    0
SEX         0
_AGEG5YR    0
MENTHLTH    0
dtype: int64

In [7]:
clean_health_df.isna().sum()

_MICHD      0.0
_RFHYPE5    0.0
TOLDHI2     0.0
_BMI5       0.0
SMOKE100    0.0
CVDSTRK3    0.0
GENHLTH     0.0
PHYSHLTH    0.0
SEX         0.0
_AGEG5YR    0.0
MENTHLTH    0.0
dtype: float64

In [8]:
renamed_df = clean_health_df.rename(columns = {'_MICHD' :'Heart Disease', '_RFHYPE5': 'High Blood Pressure', 'TOLDHI2': 'High Cholestoral',
                                   '_BMI5': 'BMI','SMOKE100': 'Smoked > 100 Cig','CVDSTRK3': 'Had Stroke','GENHLTH': 'General Health','PHYSHLTH':'Physical Health',
                                   'SEX':'SEX', '_AGE5YR': 'AGE RANGE','MENTHLTH': 'MENTAL HEALTH'})

renamed_df.head()

,Heart Disease,High Blood Pressure,High Cholestoral,BMI,Smoked > 100 Cig,Had Stroke,General Health,Physical Health,SEX,_AGEG5YR,MENTAL HEALTH


In [11]:
pip install sshtunnel

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import preprocessed (reduced, encoded, etc) data
health_df = pd.read_sql_query("select * from <table>", con=conn)
health_df.head(10)

In [ ]:
import config from config.py

conn = ps.connect(database = 'postgres',
                        user =     'postgres',
                        password = 'F1mir#spec',
                        host =     'healthcare.c4gamqekwnwh.us-west-1.rds.amazonaws.com',
                        port =     '5432')

cur = conn.cursor()

cur.execute('''
            SELECT COUNT(10)
            FROM table;
            ''')

cur.fetchall()

In [ ]:
import config


In [51]:
import psycopg2
import config as creds
# def connect():
    # Set up a connection to the postgres server.
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
              +" password="+ creds.PGPASSWORD
conn = psycopg2.connect(conn_string)
print("Connected!")
# Create a cursor object
cursor = conn.cursor()
# return conn, cursor

Connected!


In [52]:
connect()

Connected!


(<connection object at 0x000001ADD0CE8048; dsn: 'user=postgres password=xxx dbname=healthcare host=database-1.c4gamqekwnwh.us-west-1.rds.amazonaws.com port=5432', closed: 0>,
 <cursor object at 0x000001ADD009AD68; closed: 0>)

In [53]:
# Import preprocessed (reduced, encoded, etc) data
cursor.execute("select * from t2")
rows = cursor.fetchall()
print(rows)

[('1', None, None)]
